<a href="https://colab.research.google.com/github/yc386/anubis_palaeoproteomics/blob/main/train_a_RF_for_anubis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##How to train a RF model from scratch for advanced users

1. Prep training/testing datasets
2. Train a default model
3. Random and grid searches to tune hyperparameters
4. Use ROC-AUC to decide best hyperparameters

Note: Feel free to change the codes for your data. Check [scikit-learn](https://scikit-learn.org/stable/) for options and schemas.

In [ ]:
#@title import libs and functions

!pip install biopython


import pandas as pd
import regex as re
import numpy as np
from Bio.SeqUtils.IsoelectricPoint import IsoelectricPoint as IP
from scipy.stats import randint
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split, cross_val_score, KFold, StratifiedKFold
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, roc_curve, auc, classification_report, ConfusionMatrixDisplay, f1_score, roc_auc_score
import matplotlib.pyplot as plt

def extract_naked_seqs(peptide):
  return ''.join(re.findall(r'[A-Z]', peptide))

def find_patterns_v1 (peptide):
    matches = []
    for i in range(len(peptide) - 2):
        triplet = peptide[i:i+3]
        if re.match(r'[QN]', triplet[1]):
            matches.append(triplet)
    return matches


def assign_modified_class(row, deamidation_mass):
    peptide = row['Peptide']
    matched = row['matched_pattern']

    modified_pattern = re.sub(r'N|Q', rf'\g<0>[+{deamidation_mass}]', matched)

    if re.search(re.escape(modified_pattern), peptide):
        return 1
    return 0

def count_modified_pattern(row, deamidation_mass):
    peptide = row['Peptide']
    matched = row['matched_pattern']

    modified_pattern = re.sub(r'N|Q', rf'\g<0>[+{deamidation_mass}]', matched)

    return len(re.findall(re.escape(modified_pattern), peptide))


def prep_moka_for_RF (psm_file, protein_target, deamidation_mass):
  p=pd.read_table(psm_file)
  p1=p[p['Proteins'].str.contains(protein_target)]
  p1=p1.assign(seq_naked=p1['Peptide'].apply(extract_naked_seqs))
  p2 = p1['seq_naked'].apply(find_patterns_v1).explode().reset_index()
  p2.columns = ['original_index', 'matched_pattern']
  p2 = p2.merge(p1, left_on='original_index', right_index=True)
  p3=p2.fillna(0)
  p4=p3[p3['matched_pattern']!=0]
  p5=p4.assign(modified_class=p4.apply(assign_modified_class, axis=1, deamidation_mass=deamidation_mass),
               modified_count=p4.apply(count_modified_pattern, axis=1, deamidation_mass=deamidation_mass),
               pI=p4['seq_naked'].apply (lambda x: IP(x).pi()))
  p6=p5.assign(adjusted_pI=p5['pI']-7.0,
               pep_len=p5['seq_naked'].str.len(),
               FDR_adjusted=((p5['mokapot q-value']-0.01)/0.01)*100
               )
  p7=p6[['matched_pattern', 'FileName', 'ScanNr', 'Peptide', 'seq_naked', 'pI', 'adjusted_pI', 'pep_len', 'modified_class', 'modified_count', 'FDR_adjusted']]

  return p7

def prep_Sage (sage_file):
  s=pd.read_table(sage_file)
  s=s.assign(ScanNr=s['scannr'].str.extract(r'scan=(\d+)', expand=False))
  s=s.rename(columns={'filename': 'FileName'})
  s1=s[['FileName', 'ScanNr', 'ms2_intensity', 'delta_rt_model', 'matched_intensity_pct', 'missed_cleavages', 'precursor_ppm']]
  s1=s1.assign(ScanNr=s1['ScanNr'].astype(int))

  return s1

def get_combined_df (sage_file, psm_file, protein_target, protein_info, sample, deamidation_mass):
  s=prep_Sage(sage_file)
  p=prep_moka_for_RF(psm_file, protein_target, deamidation_mass)
  c=pd.merge(s, p, on=['FileName', 'ScanNr'])
  i=pd.read_csv(protein_info)
  c1=pd.merge(i, c, on='matched_pattern')
  int_median=c1['ms2_intensity'].median()
  c1=c1.assign(int_ratio=((c1['ms2_intensity']-int_median)/int_median)*100,
               tryp_ratio=(c1['pep_len']/(c1['pep_len']+c1['missed_cleavages']))*100,
               sample=sample)

  return c1


# add inputs and prep the data

In [ ]:
#path to results.sage.tsv
sage_file='path_to_sage'
#path to mokapot.psms.txt
psm_file='path_to_mokapot'
#uniprot ID of the target
protein_target='P02754'
#path to a protein info file, check get_dihedrals.ipynb
protein_info='/content/P02754 (1).csv'
#sample_name
sample='sample_name'
#deamidation mass used in the search
deamidation_mass=0.9848
#call the function to prep the data
df=get_combined_df (sage_file, psm_file, protein_target, protein_info, sample, deamidation_mass)
df

,Residue,Residue_ID,Chain,Phi,Psi,SASA,Residue_before,Residue_after,CG_N_Distance,real_position,...,seq_naked,pI,adjusted_pI,pep_len,modified_class,modified_count,FDR_adjusted,int_ratio,tryp_ratio,sample
0,Q,35,A,-48.346937,-31.406966,32.86,A,S,4.586759,51,...,VAGTWYSLAMAASDISLLDAQSAPLR,4.207757,-2.792243,26,1,1,-98.451053,52.681343,100.0,Jeong_2018
1,Q,35,A,-48.346937,-31.406966,32.86,A,S,4.586759,51,...,VAGTWYSLAMAASDISLLDAQSAPLR,4.207757,-2.792243,26,0,0,-98.451053,26.407313,100.0,Jeong_2018
2,Q,35,A,-48.346937,-31.406966,32.86,A,S,4.586759,51,...,VAGTWYSLAMAASDISLLDAQSAPLR,4.207757,-2.792243,26,0,0,-98.451053,-34.347463,100.0,Jeong_2018
3,Q,35,A,-48.346937,-31.406966,32.86,A,S,4.586759,51,...,VAGTWYSLAMAASDISLLDAQSAPLR,4.207757,-2.792243,26,0,0,-98.451053,7.322942,100.0,Jeong_2018
4,Q,35,A,-48.346937,-31.406966,32.86,A,S,4.586759,51,...,VAGTWYSLAMAASDISLLDAQSAPLR,4.207757,-2.792243,26,1,1,-97.438196,-25.247800,100.0,Jeong_2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,Q,159,A,-54.730450,126.769746,96.09,E,C,4.748623,175,...,LSFNPTQLEEQCHI,4.507184,-2.492816,14,0,0,961.905571,-43.346276,100.0,Jeong_2018
197,Q,159,A,-54.730450,126.769746,96.09,E,C,4.748623,175,...,LSFNPTQLEEQCHI,4.507184,-2.492816,14,1,1,160.628071,-69.416994,100.0,Jeong_2018
198,Q,159,A,-54.730450,126.769746,96.09,E,C,4.748623,175,...,LSFNPTQLEEQCHI,4.507184,-2.492816,14,1,1,1763.529584,-72.669751,100.0,Jeong_2018
199,Q,159,A,-54.730450,126.769746,96.09,E,C,4.748623,175,...,LSFNPTQLEEQCHI,4.507184,-2.492816,14,1,1,3512.588652,-63.847320,100.0,Jeong_2018


# Train a default RF model

In [ ]:
X=df[['Phi', 'Psi', 'SASA', 'CG_N_Distance', 'IUPRED', 'Robinson_half', 'int_ratio', 'adjusted_pI', 'FDR_adjusted', 'tryp_ratio', 'delta_rt_model', 'matched_intensity_pct', 'precursor_ppm']]
y = df['modified_class']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
model = RandomForestClassifier(random_state = 7)
model.fit(X_train, y_train)


y_pred = model.predict(X_test)


print(classification_report(y_pred, y_test))

In [ ]:
#display a confusion matrix
cm = confusion_matrix(y_test, y_pred)

ConfusionMatrixDisplay(confusion_matrix=cm).plot()

In [ ]:
#evaluate feature importance
Feature_Names = X_train.columns
importances = model.feature_importances_
f = pd.DataFrame({'Feature': Feature_Names, 'Gini Importance': importances}).sort_values('Gini Importance', ascending=False)
f

# Random searches to find hyperparameter ranges

In [ ]:
rf_params = {
    'n_estimators': randint(50,500),
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2']
}

In [ ]:
rf_random_search = RandomizedSearchCV(estimator=RandomForestClassifier(random_state=42),
                                      param_distributions=rf_params, n_iter=100, cv=5, random_state=42,
                                      scoring='f1',
                                      return_train_score=True)
rf_random_search.fit(X_train, y_train)

In [ ]:
rf_random_search.best_params_

In [ ]:
#to save the tuning results
tuning_result_rf_rs = pd.DataFrame(rf_random_search.cv_results_)
tuning_result_rf_rs

In [ ]:
y_pred_rf_rs = rf_random_search.predict(X_test)
print("Classification Report:")
print(classification_report(y_test, y_pred_rf_rs))

# grid searches for precise tuning

In [ ]:
param_grid = {
    'max_depth': [15, 20, 25],
    'n_estimators': [350, 375, 400]
}

In [ ]:
grid_search = GridSearchCV(estimator=RandomForestClassifier(random_state=42),
                                      param_grid=param_grid, cv=5,
                                      scoring='f1')
grid_search.fit(X_train, y_train)

In [ ]:
grid_search.best_params_

# ROC-AUC curves to decide the best model

In [ ]:
default_y=model.predict_proba(X_test)
random_y=rf_random_search.predict_proba(X_test)
grid_y = grid_search.predict_proba(X_test)

In [ ]:
test_fpr, test_tpr, test_thresholds = roc_curve(y_true=y_test, y_score=default_y[:,1])
random_fpr, random_tpr, random_thresholds = roc_curve(y_true=y_test, y_score=random_y[:,1])
grid_fpr, grid_tpr, grid_thresholds = roc_curve(y_true=y_test, y_score=grid_y[:,1])

In [ ]:
test_auc = roc_auc_score(y_test, default_y[:, 1])
random_auc = roc_auc_score(y_test, random_y[:, 1])
grid_auc = roc_auc_score(y_test, grid_y[:, 1])

In [ ]:
plt.figure(figsize=(6, 6))
plt.plot(random_fpr, random_tpr, label=f'Random Search (AUC= {random_auc:.4f})', color='#8a5dee')
plt.plot(grid_fpr, grid_tpr, label=f'Grid Search (AUC = {grid_auc:.4f})', color='#3382e7')
plt.plot(test_fpr, test_tpr, label=f'Default (AUC = {test_auc:.4f})', color='#7cbf3b')
plt.plot([0, 1], [0, 1], color = 'black', linewidth = 2, linestyle = '--', label='random classifier, AUC=0.5')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend()
plt.title('Receiver Operating Characteristic (ROC)')
plt.show()